In [ ]:
import torch
import torch.nn as nn
import math
import re

In [ ]:
def preprocess_game(game_str):
    moves = game_str.split()
    input_output_pairs = []
    for i in range(1, len(moves)):
        input_seq = ' '.join(moves[:i])
        output_move = moves[i]
        input_output_pairs.append((input_seq, output_move))
    return input_output_pairs

def preprocess_file(file_path, game_count=float('inf')):
    all_pairs = []
    processed_game_count = 0
    
    with open(file_path, 'r') as file:
        for line in file:
            game = line.strip()
            all_pairs.extend(preprocess_game(game))
            processed_game_count += 1
            if processed_game_count >= game_count:
                break
    
    return all_pairs

file_path = 'out/grandmaster.txt'
training_data = preprocess_file(file_path, 100)
len(training_data)

In [ ]:
training_data[:10]

In [ ]:
vocab = set()

files = [
#     'out/beginner.txt',
#     'out/intermediate.txt',
#     'out/master.txt',
    'out/grandmaster.txt',
]

for file_path in files:
    with open(file_path, 'r') as file:
        for line in file:
            for word in line.split(' '):
                vocab.add(word.strip())
vocab = sorted(list(vocab))
print(len(vocab))

In [ ]:
with open('vocab.txt', 'w') as file:
    file.write('\n'.join(vocab))

In [ ]:
vocab = open('vocab.txt', 'r').read().splitlines()
vocab

In [ ]:
vocab[:30]

In [ ]:
# Assuming 'vocab' is your list of moves as created in your code snippet
move_to_index = {move: idx for idx, move in enumerate(vocab)}
index_to_move = {idx: move for idx, move in enumerate(vocab)}

# Add special tokens
UNK_TOKEN = '<UNK>'
PAD_TOKEN = '<PAD>'
move_to_index[UNK_TOKEN] = len(move_to_index)
move_to_index[PAD_TOKEN] = len(move_to_index)
index_to_move[len(index_to_move)] = UNK_TOKEN
index_to_move[len(index_to_move)] = PAD_TOKEN

def encode_move(move):
    return move_to_index.get(move, move_to_index[UNK_TOKEN])

def decode_move(index):
    return index_to_move.get(index, UNK_TOKEN)

In [ ]:
encode_move('Qe4#')

In [ ]:
decode_move(2714)

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        # Create a matrix of shape (max_len, d_model) filled with zeros
        # This will store the positional encodings for each position and dimension
        pe = torch.zeros(max_len, d_model)
        
        # Create a vector of positions from 0 to max_len-1
        # Unsqueeze to shape (max_len, 1) for broadcasting
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        
        # Create the division term for the sinusoidal function
        # This creates a vector of values that increase exponentially
        # We use log(10000.0) as it's a common choice that works well in practice
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        
        # Apply sine to even indices in the positional encoding
        # This creates a sinusoidal pattern that varies at different frequencies
        pe[:, 0::2] = torch.sin(position * div_term)
        
        # Apply cosine to odd indices in the positional encoding
        # This creates a cosinusoidal pattern that varies at different frequencies
        pe[:, 1::2] = torch.cos(position * div_term)
        
        # Unsqueeze and transpose to shape (1, max_len, d_model)
        # This allows for easy addition to the input embeddings later
        pe = pe.unsqueeze(0).transpose(0, 1)
        
        # Register the positional encoding as a buffer
        # This means it won't be considered a model parameter (won't be updated during training)
        # but will be saved and loaded with the model
        self.register_buffer('pe', pe)

    def forward(self, x):
        # Add the positional encoding to the input
        # x is expected to have shape (seq_len, batch_size, d_model)
        # We slice the positional encoding to match the input sequence length
        return x + self.pe[:x.size(0), :]

In [ ]:
class ChessTransformer(nn.Module):
    def __init__(self, vocab_size, d_model, nhead, num_encoder_layers):
        super().__init__()
        
        # Create an embedding layer to convert input tokens to vectors
        # vocab_size is the number of unique tokens in our vocabulary
        # d_model is the dimensionality of the embedding space
        self.embedding = nn.Embedding(vocab_size, d_model)
        
        # Create a positional encoding layer
        # This adds information about the position of each token in the sequence
        self.pos_encoder = PositionalEncoding(d_model)
        
        # Create a single transformer encoder layer
        # This includes self-attention and feedforward neural network
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead)
        
        # Create the full transformer encoder by stacking multiple encoder layers
        # num_encoder_layers determines the depth of the network
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_encoder_layers)
        
        # Store d_model for use in the forward pass
        self.d_model = d_model
        
        # Create a linear layer for the final output
        # This projects the transformer output back to vocabulary space
        self.linear = nn.Linear(d_model, vocab_size)

    def forward(self, src):
        # Convert input tokens to embeddings
        # Multiply by sqrt(d_model) to scale the embeddings
        # This scaling helps maintain the variance of the forward pass
        src = self.embedding(src) * math.sqrt(self.d_model)
        
        # Add positional encoding to the embeddings
        src = self.pos_encoder(src)
        
        # Pass the encoded input through the transformer encoder
        output = self.transformer_encoder(src)
        
        # Project the transformer output to vocabulary space
        output = self.linear(output)
        
        return output

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class ChessDataset(Dataset):
    def __init__(self, data, max_seq_length=100):
        self.data = data
        self.max_seq_length = max_seq_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_seq, target_move = self.data[idx]
        
        # Ensure input_seq is a list
        input_seq = input_seq.split() if isinstance(input_seq, str) else input_seq
        
        # Truncate or pad input sequence to exactly max_seq_length
        if len(input_seq) > self.max_seq_length:
            input_seq = input_seq[-self.max_seq_length:]
        else:
            input_seq = [PAD_TOKEN] * (self.max_seq_length - len(input_seq)) + input_seq
        
        # Ensure the sequence is exactly max_seq_length
        input_seq = input_seq[:self.max_seq_length]
        
        # Encode moves
        input_tensor = torch.tensor([encode_move(m) for m in input_seq])
        target_tensor = torch.tensor(encode_move(target_move))
        
        return input_tensor, target_tensor

In [ ]:
training_data_size = int(0.9 * len(training_data))

# Create dataset and dataloader
dataset = ChessDataset(training_data[:training_data_size], max_seq_length=100)
dataloader = DataLoader(dataset, batch_size=256, shuffle=True, num_workers=4)

In [ ]:
# Training parameters
vocab_size = len(vocab) + 500  # Slightly larger than the actual vocabulary size                                                    (1/5 results) [1919/2674]
d_model = 512                                                                                                                                     
nhead = 8
num_encoder_layers = 6

In [ ]:
import torch.optim as optim
import gc

# Model definition
model = ChessTransformer(vocab_size, d_model, nhead, num_encoder_layers)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
    print(f"CUDA available: {torch.cuda.is_available()}")
    print(f"Current device: {torch.cuda.current_device()}")
    print(f"Device name: {torch.cuda.get_device_name()}")
if torch.backends.mps.is_available():
    device = "mps"
    print(f"MPS available: {torch.backends.mps.is_available()}")

device = torch.device(device)
model.to(device)
device

In [ ]:
import torch
import torch.nn.functional as F
import math

# Calculate the loss of a model that provides random guesses. This gives us a basis
# to measure the actual loss against while training.

def calculate_random_guess_loss(dataloader, vocab_size, device):
    total_loss = 0
    total_samples = 0
    
    with torch.no_grad():
        for batch_input, batch_target in dataloader:
            batch_size = batch_target.size(0)
            total_samples += batch_size
            
            # Create a tensor of uniform probabilities
            random_logits = torch.full((batch_size, vocab_size), 1/vocab_size, device=device)
            
            # Convert to log probabilities
            random_log_probs = torch.log(random_logits)
            
            # Calculate cross-entropy loss
            loss = F.nll_loss(random_log_probs, batch_target.to(device))
            
            total_loss += loss.item() * batch_size

    avg_loss = total_loss / total_samples
    return avg_loss

# Usage
vocab_size = len(move_to_index)  # Assuming you have this mapping from earlier
random_loss = calculate_random_guess_loss(dataloader, vocab_size, device)
print(f"Random guess loss: {random_loss:.4f}")

# Theoretical calculation
theoretical_random_loss = -math.log(1/vocab_size)
print(f"Theoretical random guess loss: {theoretical_random_loss:.4f}")

In [ ]:
from tqdm import tqdm
import time
import gc

def train_model(model, dataloader, criterion, optimizer, num_epochs, device, log_interval=100, save_interval=1000):
    model.to(device)
    total_steps = len(dataloader)
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        epoch_loss = 0.0
        start_time = time.time()
        
        progress_bar = tqdm(enumerate(dataloader), total=total_steps, desc=f"Epoch {epoch+1}/{num_epochs}")
        
        for i, (inputs, targets) in progress_bar:
            inputs, targets = inputs.to(device), targets.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs[:, -1, :], targets)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            epoch_loss += loss.item()
            
            if (i + 1) % log_interval == 0:
                avg_loss = running_loss / log_interval
                progress_bar.set_postfix({'Loss': f'{avg_loss:.4f}'})
                running_loss = 0.0
            
            if (i + 1) % save_interval == 0:
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': loss,
                }, f'checkpoint_epoch{epoch+1}_step{i+1}.pth')
        
        epoch_time = time.time() - start_time
        epoch_avg_loss = epoch_loss / total_steps
        print(f"Epoch {epoch+1}/{num_epochs} completed in {epoch_time:.2f}s - Avg Loss: {epoch_avg_loss:.4f}")
        print(f"GPU memory allocated: {torch.cuda.memory_allocated()/1e9}GB")
        print(f"GPU memory cached: {torch.cuda.memory_cached()/1e9}GB")

In [ ]:
num_epochs = 10
log_interval = 100  # Log every 100 batches
save_interval = 1000  # Save checkpoint every 1000 batches

print("Starting training loop...")
total_start_time =- time.perf_counter()

train_model(model, dataloader, criterion, optimizer, num_epochs, device, log_interval, save_interval)

total_end_time = time.perf_counter()
total_duration = total_end_time - total_start_time
print(f"Total training time: {total_duration:.2f} seconds")

In [ ]:
# Inference function
def predict_next_move(model, move_sequence):
    model.eval()
    with torch.no_grad():
        input_tensor = torch.tensor([encode_move(m) for m in move_sequence.split()]).unsqueeze(0).to(device)
        output = model(input_tensor)
        predicted_move_index = output[0, -1, :].argmax().item()
        return decode_move(predicted_move_index)

# Example usage
game_so_far = "e4"
next_move = predict_next_move(model, game_so_far)
print(f"Predicted next move: {next_move}")